## Import library

In [1]:
import os
import sys
lib_path = os.path.abspath("").replace("notebooks", "src")
sys.path.append(lib_path)
import torch
import pandas
from data.dataloader import TorchDataset
from torch.utils.data import DataLoader
from utils.torch.trainer import TorchTrainer
import torchvision.transforms as transforms
from torch import nn
from utils.torch.callbacks import CheckpointsCallback

2023-04-27 23:22:17.906588: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Hyperparameters

In [2]:
batch_size = 32
im_size = (224,224)
epochs=5

## Dataset

In [3]:
train = pandas.read_csv("train-scene-classification/train.csv")
images_name = train["image_name"].tolist()
labels = train["label"].tolist()
images_name = ["train-scene-classification/train/" + x for x in images_name]
labels = [int(x) for x in labels]
num_classes = len(set(labels))

In [4]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = TorchDataset(images_name[:-128], labels[:-128], im_size=im_size, transform=transform)
test_data = TorchDataset(images_name[-128:], labels[-128:], im_size=im_size, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)

## Model

In [5]:
loss_fn = torch.nn.CrossEntropyLoss()

class SimpleClassification(TorchTrainer):
    def __init__(self, num_classes, im_size=(224,224), **kwargs):
        super(SimpleClassification, self).__init__(**kwargs)
        self.network = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(im_size[0]//8 * im_size[1]//8 * 64, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    def forward(self, x):
        return self.network(x)
    
    def train_step(self, batch):
        self.optimizer.zero_grad()
        
        inputs, labels = batch["inputs"], batch["labels"]
        
        logits = self(inputs)
        
        loss = loss_fn(logits, labels)
    
        loss.backward()
        
        self.optimizer.step()
        
        acc = (logits.argmax(dim=1) == labels).float().mean()
        
        return {"loss": loss.item(), "acc": acc.item()}
    def test_step(self, batch):
        inputs, labels = batch["inputs"], batch["labels"]
        
        logits = self(inputs)
        
        loss = loss_fn(logits, labels)
        
        acc = (logits.argmax(dim=1) == labels).float().mean()
        
        return {"loss": loss.item(), "acc": acc.item()}
    
model = SimpleClassification(num_classes, im_size=im_size)

## Training

In [6]:
ckpt_callback = CheckpointsCallback("checkpoints", save_freq=1000, keep_one_only=True)
model.compile(optimizer="sgd")
model.fit(train_dataloader, epochs=epochs, callbacks=[ckpt_callback])

2023-04-27 23:22:21,249 - Training - INFO - Epoch 1/5
loss: 1.5706 acc: 0.5312 :  11%|█         | 58/530 [00:26<03:33,  2.21it/s]


KeyboardInterrupt: 

## Testing

In [ ]:
model = SimpleClassification.load("checkpoints/checkpoint_.pt")
y_true = []
y_pred = []
for batch in test_dataloader:
    inputs, labels = batch["inputs"], batch["labels"]
    logits = model.predict(inputs)
    y_true.append(labels.item())
    y_pred.append(logits.argmax(dim=1).item())

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))